In [1]:
#導入所需套件
import pandas as pd
import numpy as np
import os

path='Train_DataSet\\'
file_path=pd.read_excel("Train_Quality_DataSet\\quality_size.xlsx")['key']   #將file夾名稱放進file_path
right_list=list()
left_list=list()

#使用for迴圈設定完整的.xlsx路徑，讀進raw data後放入right_list及left_list
for i in file_path:
    dat_path=path+"file_"+str(i)
#     print(dat_path)
    excel_path=os.listdir(dat_path)
    for j in excel_path:
        if j=='pressure.xlsx':
            #print(dat_path+"\\"+j)
            A=pd.read_excel(dat_path+"\\"+j)
            A=A.drop(['Unnamed: 0','TimeStamp'], axis=1)
#               print(A.shape)                        
            for k in A:
                if k=='pressure_right':
                    right_list.append([A[k]])
                elif k=='pressure_left':
                    left_list.append([A[k]])
print("right_list_shape:",str(np.array(right_list).shape))
print("left_list_shape:",str(np.array(left_list).shape))

right_list_shape: (159, 1)
left_list_shape: (159, 1)


In [2]:
#導入所需套件
from keras.preprocessing import sequence

#使用pad_sequences進行data補值，並轉換list維度
cnt=0
for i in right_list:
    right_list[cnt]=sequence.pad_sequences(i,dtype='float64',maxlen=7443)          
    cnt+=1
print("right_list_shape:",str(np.array(right_list).shape))                                  
right_list = np.reshape(right_list,(-1,7443))                           #三維轉二維
print("right_list_shape:",str(np.array(right_list).shape))              

cnt=0
for i in left_list:
    left_list[cnt]=sequence.pad_sequences(i,dtype='float64',maxlen=7443)          
    cnt+=1
print("left_list_shape:",str(np.array(left_list).shape))                                  
left_list = np.reshape(left_list,(-1,7443))                           #三維轉二維
print("left_list_shape:",str(np.array(left_list).shape))      

Using TensorFlow backend.


right_list_shape: (159, 1, 7443)
right_list_shape: (159, 7443)
left_list_shape: (159, 1, 7443)
left_list_shape: (159, 7443)


In [3]:
#導入所需套件
from sklearn import preprocessing

#對data進行正規化計算
right_list=right_list/right_list.max()
print("right_list[158]:",right_list[158])

left_list=left_list/left_list.max()
print("left_list[158]:",left_list[158])

right_list[158]: [0.         0.         0.         ... 0.00929615 0.00929615 0.00929615]
left_list[158]: [0.         0.         0.         ... 0.00837989 0.00837989 0.00837989]


In [4]:
#導入所需套件
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

#讀進quality_size.xlsx
path='Train_Quality_DataSet\\quality_size.xlsx'
PN=pd.read_excel(path)

#labels設定
num_of_samples=right_list.shape[0]
right_labels=np.zeros((num_of_samples,),dtype='float64')
left_labels=np.zeros((num_of_samples,),dtype='float64')
cnt=0
for i in PN['right']:
    right_labels[cnt]=i
    cnt+=1
print("right_labels_shape:",right_labels.shape)
print("right_labels:",right_labels)

cnt=0
for i in PN['left']:
    left_labels[cnt]=i
    cnt+=1
print("left_labels_shape:",left_labels.shape)
print("left_labels:",left_labels)

#使用shuffle對feature、labels進行亂數配對
#切割訓練集成訓練集及驗證集
x_right,y_right=shuffle(right_list,right_labels,random_state=2)
x_right_train,x_right_test,y_right_train,y_right_test_=train_test_split(x_right,y_right,test_size=0.2,random_state=2)

x_left,y_left=shuffle(left_list,left_labels,random_state=2)
x_left_train,x_left_test,y_left_train,y_left_test=train_test_split(x_left,y_left,test_size=0.2,random_state=2)

right_labels_shape: (159,)
right_labels: [36.14 36.16 36.18 36.18 36.2  36.22 36.15 36.28 36.16 36.15 36.19 36.17
 36.14 36.22 36.16 36.17 36.16 36.2  36.18 36.16 36.23 36.19 36.13 36.16
 36.14 36.14 36.16 36.18 36.18 36.17 36.16 36.15 36.15 36.16 36.14 36.16
 36.15 36.16 36.11 36.19 36.14 36.16 36.2  36.16 36.13 36.17 36.18 36.16
 36.14 36.2  36.07 36.18 36.18 36.15 36.15 36.2  36.04 36.08 36.16 36.2
 36.1  36.16 36.2  36.16 36.18 36.18 36.19 36.16 36.13 36.17 36.19 36.18
 36.15 36.14 36.11 36.16 36.18 36.22 36.18 36.2  36.19 36.17 36.18 36.19
 36.13 36.16 36.21 36.18 36.25 36.16 36.22 36.2  36.23 36.1  36.18 36.12
 36.14 36.12 36.16 36.17 36.18 36.17 36.2  36.17 36.17 36.18 36.12 36.1
 36.17 36.16 36.17 36.18 36.18 36.05 36.18 36.23 36.07 36.18 36.19 36.2
 36.19 36.19 36.19 36.17 36.21 36.2  36.19 36.18 36.15 36.17 36.14 36.21
 36.14 36.24 36.19 36.17 36.21 36.13 36.18 36.16 36.17 36.17 36.15 36.2
 36.17 36.17 36.13 36.14 36.15 36.17 36.21 36.21 36.03 36.2  36.16 36.13
 36.16 36.14 3

In [6]:
#導入所需套件
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
    
#使用LinearRegression()訓練並預測
reg1 = LinearRegression()
reg1.fit(x_right_train,y_right_train)
predict1=reg1.predict(x_right_test)
print("RMSE of Rihgt test size predict:",np.sqrt(mean_squared_error(y_right_test_,predict1)))
reg1.fit(x_right,y_right)

reg2 = LinearRegression()
reg2.fit(x_left_train,y_left_train)
predict2=reg2.predict(x_left_test)
print("RMSE of Left test size predict:",np.sqrt(mean_squared_error(y_left_test,predict2)))
reg2.fit(x_left,y_left)

RMSE of Rihgt test size predict: 0.03460868071088808
RMSE of Left test size predict: 0.05389671432651268


In [7]:
# chcek test predict size

answer=reg1.predict(x_left_test)
print(answer[0:5])
print(y_left_test[0:5])

[36.2383241  36.16258574 36.20248923 36.15986438 36.21061447 36.17082048
 36.2045835  36.17805701 36.15541594 36.1506122  36.14163379 36.19171919
 36.16951222 36.17351221 36.23496621 36.14676579 36.23844244 36.16078304
 36.18435734 36.17997768 36.16343414 36.21929425 36.14877493 36.20890845
 36.1329762  36.19795502 36.11827624 36.1580255  36.15343762 36.16040882
 36.14609914 36.19200609 36.15564855 36.16737171 36.16401511 36.16866416
 36.16753667 36.16218328 36.20419671]
[36.19 36.19 36.19 36.17 36.21 36.2  36.19 36.18 36.15 36.17 36.14 36.21
 36.14 36.24 36.19 36.17 36.21 36.13 36.18 36.16 36.17 36.17 36.15 36.2
 36.17 36.17 36.13 36.14 36.15 36.17 36.21 36.21 36.03 36.2  36.16 36.13
 36.16 36.14 36.15]


In [7]:
#導入所需套件
from sklearn.externals import joblib

#model儲存
joblib.dump(reg1,'right_size.pkl')
joblib.dump(reg2,'left_size.pkl')

D:\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['left_size.pkl']